# Environment configuration

In [1]:
import findspark
findspark.init('/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession 
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from matplotlib import pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Load data from the file

In [3]:
rdd = sc.textFile('./data/ratings_Sports_and_Outdoors.csv')
rdd = rdd.map(lambda x: x.split(','))
rdd.take(10)

[[u'A3PMSRCL80KSA1', u'0000031852', u'4.0', u'1388275200'],
 [u'A1SNLWGLFXD70K', u'0000031852', u'4.0', u'1392940800'],
 [u'A1KJ4CVG87QW09', u'0000031852', u'4.0', u'1389657600'],
 [u'AA9ITO6ZLZW6', u'0000031852', u'5.0', u'1399507200'],
 [u'APJ5ULJ1RMZ4', u'0000031852', u'1.0', u'1398556800'],
 [u'A2PAVURT4NOHE1', u'0000031852', u'5.0', u'1388361600'],
 [u'A3URQ0LXLV46E9', u'0000031852', u'4.0', u'1400544000'],
 [u'A2681T699HV6H1', u'0000031895', u'4.0', u'1384905600'],
 [u'A2EPSZKEG06QZE', u'0000031895', u'2.0', u'1396224000'],
 [u'A23K73OVXJ04EG', u'0000031895', u'5.0', u'1391212800']]

# Label users and items
## The function merges 2 dictionaries

In [3]:
def mergedic(x, y):
    x.update(y)
    return x

## Create RDDs

In [4]:
def count_plot(x, filename):
    value = map(lambda a: a[0], x)
    label = map(lambda a: a[1], x)
    plt.figure(figsize = (15,8))
    plt.barh(range(len(value)), value, color = 'blue')
    plt.yticks(range(len(value)), label)
    plt.savefig(filename)

In [4]:
user = rdd.map(lambda x: x[0])
num_user = user.distinct().count()
print num_user
#user.take(10)

3195


In [6]:
user_rnum_count = user.map(lambda x: (x, 1))\
                  .reduceByKey(lambda x, y: x + y)\
                  .map(lambda x: (x[1], x[0]))\
                  .sortByKey(False)
count_plot(user_rnum_count.take(15), 'usernum.png')

In [5]:
item = rdd.map(lambda x: x[1])
num_item = item.distinct().count()
print num_item
#item.take(10)

2990


In [8]:
item_rnum_count = item.map(lambda x: (x, 1))\
                  .reduceByKey(lambda x, y: x + y)\
                  .map(lambda x: (x[1], x[0]))\
                  .sortByKey(False)
count_plot(item_rnum_count.take(15), 'itemnum.png')

In [9]:
rating_count = rdd.map(lambda x: (x[2], 1))\
               .reduceByKey(lambda x, y: x + y)\
               .map(lambda x: (x[1], x[0]))
#rating_count.take()
count_plot(rating_count.collect(), 'ratingnum.png')

In [10]:
unrdd = sc.parallelize(range(1, num_user + 1))
inrdd = sc.parallelize(range(1, num_item + 1))
pair = unrdd.cartesian(inrdd)
#pair.take(10)

## Create dictionaries

In [6]:
userd = user.distinct().zipWithIndex()
userd = userd.map(lambda x: {x[0]: x[1] + 1})
user_dic = userd.reduce(mergedic)

In [7]:
itemd = item.distinct().zipWithIndex()
itemd = itemd.map(lambda x: {x[0]: x[1] + 1})
item_dic = itemd.reduce(mergedic)

In [13]:
cb = rdd.map(lambda (user, item, rating, timestamp): [user, item, rating, user_dic[user], item_dic[item]])
#cb.take(5)

# Create dataframes

In [14]:
cbrdd = cb.map(lambda x: Row(username = x[0],
                             itemname = x[1],
                             rating = float(x[2]),
                             userid = int(x[3]),
                             itemid = int(x[4])))
#cbrdd.take(5)

In [15]:
df = spark.createDataFrame(cbrdd)
df.repartition(1).write.csv('tempcsv', sep = ',')

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 43, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/lmh/Programs/spark-2.2.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ip

In [18]:
meandf = df.groupby('userid').agg({'rating': 'mean'})
#meandf = df.join(tdf, df.userid == tdf.userid).drop(tdf.userid)
#meandf = meandf.withColumnRenamed('avg(rating)', 'mean')
meandf.repartition(1).write.csv('tempcsv')